In [1]:
# install dependencies
!pip install fuzzy
!pip install rapidfuzz
!pip install ace_tools

  Preparing metadata (setup.py) ... done
  Created wheel for fuzzy: filename=Fuzzy-1.2.2-cp311-cp311-linux_x86_64.whl size=220705 sha256=990c65ac369f169c4f6ed69e0c7c4ff9645fb7b96a0273d81d331f9b3057263d
  Stored in directory: /root/.cache/pip/wheels/c7/1c/77/28af87176ebf6eb6208c17e64a45a8e48eda4194bd8f605096
Successfully built fuzzy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.6 MB/s eta 0:00:00


In [3]:
# perform cleaning/preprocessing to merge datasets
import pandas as pd

# load data
df1 = pd.read_csv('/content/player_info.csv')  # Original player info
df2 = pd.read_csv('/content/player_data_for_positions.csv')  # Supplemental info

# normalize for cleaning
df1_clean = df1.copy()
df2_clean = df2.copy()

# standardize datasets for merging
df1_clean = df1_clean.rename(columns={
    'playerName': 'player_name',
    'From': 'year_start',
    'To': 'year_end',
    'Pos': 'position',
    'Ht': 'height',
    'Wt': 'weight',
    'birthDate': 'birth_date',
    'Colleges': 'college'
})

df2_clean = df2_clean.rename(columns={
    'name': 'player_name'
})

# standardize name formatting
df1_clean['player_name'] = df1_clean['player_name'].str.strip()
df2_clean['player_name'] = df2_clean['player_name'].str.strip()

# merge on player_name, prioritizing df1 (player_info) and updating with df2 if missing
merged_players = pd.merge(df1_clean, df2_clean, on='player_name', how='outer', suffixes=('_info', '_pos'))

# fill missing fields in df1 from df2 where possible
for col in ['year_start', 'year_end', 'position', 'height', 'weight', 'birth_date', 'college']:
    info_col = f"{col}_info"
    pos_col = f"{col}_pos"
    if info_col in merged_players.columns and pos_col in merged_players.columns:
        merged_players[col] = merged_players[info_col].combine_first(merged_players[pos_col])
        merged_players.drop(columns=[info_col, pos_col], inplace=True)

In [4]:
# change floats dtype to int
cols_to_convert = ['year_start', 'year_end', 'weight']
for col in cols_to_convert:
    merged_players[col] = pd.to_numeric(merged_players[col], errors='coerce').dropna().astype(int)

In [5]:
# get career length
merged_players["career_length"] = merged_players["year_end"] - merged_players["year_start"] + 1

In [6]:
merged_players

,player_name,year_start,year_end,position,height,weight,birth_date,college,career_length
0,A.C. Green,1986,2001,F-C,6-9,220.0,"October 4, 1963",Oregon State,16
1,A.J. Bramlett,2000,2000,C,6-10,227.0,"January 10, 1977",Arizona,1
2,A.J. English,1991,1992,G,6-3,175.0,"July 11, 1967",Virginia Union University,2
3,A.J. Guyton,2001,2003,G,6-1,180.0,"February 12, 1978",Indiana,3
4,A.J. Hammons,2017,2017,C,7-0,260.0,"August 27, 1992",Purdue,1
...,...,...,...,...,...,...,...,...,...
5415,Šarūnas Marčiulionis*,1990,1997,G,6-5,200.0,"June 13, 1964",NaN,8
5416,Žan Tabak,1995,2001,C,7-0,245.0,"June 15, 1970",NaN,7
5417,Žarko Paspalj,1990,1990,F,6-9,215.0,"March 27, 1966",NaN,1
5418,Žarko Čabarkapa,2004,2006,F,6-11,235.0,"May 21, 1981",NaN,3


In [7]:
# drop unnecessary cols
players_df = merged_players.drop(columns=['year_start'])
players_df


,player_name,year_end,position,height,weight,birth_date,college,career_length
0,A.C. Green,2001,F-C,6-9,220.0,"October 4, 1963",Oregon State,16
1,A.J. Bramlett,2000,C,6-10,227.0,"January 10, 1977",Arizona,1
2,A.J. English,1992,G,6-3,175.0,"July 11, 1967",Virginia Union University,2
3,A.J. Guyton,2003,G,6-1,180.0,"February 12, 1978",Indiana,3
4,A.J. Hammons,2017,C,7-0,260.0,"August 27, 1992",Purdue,1
...,...,...,...,...,...,...,...,...
5415,Šarūnas Marčiulionis*,1997,G,6-5,200.0,"June 13, 1964",NaN,8
5416,Žan Tabak,2001,C,7-0,245.0,"June 15, 1970",NaN,7
5417,Žarko Paspalj,1990,F,6-9,215.0,"March 27, 1966",NaN,1
5418,Žarko Čabarkapa,2006,F,6-11,235.0,"May 21, 1981",NaN,3


In [8]:
save_path = '/content/players_df.csv'
players_df.to_csv(save_path, index=False)

In [29]:
import pandas as pd
from rapidfuzz import process, fuzz

# Load datasets
players_df = pd.read_csv('/content/players_df.csv')
injury_df = pd.read_csv('/content/cleaned_data.csv')

# Clean player names
players_df['player_name'] = players_df['player_name'].str.strip()
injury_df['player_name'] = injury_df['player_name'].str.strip()

# Fuzzy match player names
known_names = players_df['player_name'].tolist()
fuzzy_matches = []
for name in injury_df['player_name'].unique():
    best_match, score, _ = process.extractOne(name, known_names, scorer=fuzz.WRatio)
    fuzzy_matches.append({
        'original_name': name,
        'matched_name': best_match,
        'score': score
    })

# Merge fuzzy match results
match_df = pd.DataFrame(fuzzy_matches)
injury_df = injury_df.merge(match_df, left_on='player_name', right_on='original_name', how='left')

# Merge player metadata
injury_df = injury_df.merge(players_df, left_on='matched_name', right_on='player_name', how='left', suffixes=('', '_player'))

# Handle overlapping columns
if 'year_end_player' in injury_df.columns and 'year_end' not in injury_df.columns:
    injury_df['year_end'] = injury_df['year_end_player']
    injury_df.drop(columns=['year_end_player'], inplace=True)

# Filter matches and drop fuzzy matching metadata
injury_df['is_match'] = injury_df['score'] >= 88
cleaned_injury_df = injury_df[injury_df['is_match'] == True].copy()
cleaned_injury_df.drop(columns=['score', 'is_match', 'player_name', 'matched_name', 'original_name'], inplace=True)

cleaned_injury_df


,notes_clean,injured,activated,days_injured,injury_type,injury_duration,player_name_player,year_end,position,height,weight,birth_date,college,career_length
0,placed on disabled list with knee injury,1962-11-14,NaN,NaN,joint,unknown,Al Ferrari,1963,G-F,6-4,190.0,"July 6, 1933",Michigan State,8
1,placed on il with sprained ankle,1969-10-15,1969-10-28,13.0,joint,medium,Bob Greacen,1972,F,6-7,206.0,"September 15, 1947",Rutgers University,3
2,placed on il with fractured right hand,1973-11-17,1973-12-20,33.0,bone,medium,Mike D'Antoni,1977,G,6-3,185.0,"May 8, 1951",Marshall,4
3,placed on il with hairline fracture below left...,1976-01-17,1976-02-16,30.0,bone,medium,Ron Behagen,1980,F-C,6-9,185.0,"January 14, 1951",Minnesota,7
4,placed on il with right knee inflammation,1984-10-23,NaN,NaN,joint,unknown,Foots Walker,1984,G,6-0,184.0,"May 21, 1951","Vincennes University, West Georgia",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127972,placed on il with sore left knee,2021-12-11,2021-12-13,2.0,joint,short,Jaren Jackson Jr.,2022,F,6-11,242.0,"September 15, 1999",Michigan State,4
127973,placed on il with recovering from surgery on r...,2022-10-17,2022-11-15,29.0,bone,medium,Jaren Jackson Jr.,2022,F,6-11,242.0,"September 15, 1999",Michigan State,4
127974,placed on il with right calf injury sore righ...,2021-12-17,2021-12-31,14.0,joint,medium,Santi Aldama,2022,F,6-11,224.0,"January 10, 2001",Loyola (MD),1
127975,placed on il with sore right knee out for season,2022-04-19,2022-11-27,222.0,joint,long,Santi Aldama,2022,F,6-11,224.0,"January 10, 2001",Loyola (MD),1


In [30]:
cleaned_injury_df = cleaned_injury_df.rename(columns={'player_name_player':'player_name'})
cleaned_injury_df

,notes_clean,injured,activated,days_injured,injury_type,injury_duration,player_name,year_end,position,height,weight,birth_date,college,career_length
0,placed on disabled list with knee injury,1962-11-14,NaN,NaN,joint,unknown,Al Ferrari,1963,G-F,6-4,190.0,"July 6, 1933",Michigan State,8
1,placed on il with sprained ankle,1969-10-15,1969-10-28,13.0,joint,medium,Bob Greacen,1972,F,6-7,206.0,"September 15, 1947",Rutgers University,3
2,placed on il with fractured right hand,1973-11-17,1973-12-20,33.0,bone,medium,Mike D'Antoni,1977,G,6-3,185.0,"May 8, 1951",Marshall,4
3,placed on il with hairline fracture below left...,1976-01-17,1976-02-16,30.0,bone,medium,Ron Behagen,1980,F-C,6-9,185.0,"January 14, 1951",Minnesota,7
4,placed on il with right knee inflammation,1984-10-23,NaN,NaN,joint,unknown,Foots Walker,1984,G,6-0,184.0,"May 21, 1951","Vincennes University, West Georgia",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127972,placed on il with sore left knee,2021-12-11,2021-12-13,2.0,joint,short,Jaren Jackson Jr.,2022,F,6-11,242.0,"September 15, 1999",Michigan State,4
127973,placed on il with recovering from surgery on r...,2022-10-17,2022-11-15,29.0,bone,medium,Jaren Jackson Jr.,2022,F,6-11,242.0,"September 15, 1999",Michigan State,4
127974,placed on il with right calf injury sore righ...,2021-12-17,2021-12-31,14.0,joint,medium,Santi Aldama,2022,F,6-11,224.0,"January 10, 2001",Loyola (MD),1
127975,placed on il with sore right knee out for season,2022-04-19,2022-11-27,222.0,joint,long,Santi Aldama,2022,F,6-11,224.0,"January 10, 2001",Loyola (MD),1


In [31]:
cleaned_injury_df.isnull().sum()

,0
notes_clean,0
injured,0
activated,1885
days_injured,1885
injury_type,0
injury_duration,0
player_name,0
year_end,0
position,0
height,0


In [32]:
cleaned_injury_df['college'] = cleaned_injury_df['college'].fillna('Skipped college')

In [34]:
import pandas as pd

# Ensure both columns are datetime
cleaned_injury_df["birth_date"] = pd.to_datetime(cleaned_injury_df["birth_date"], errors="coerce")
cleaned_injury_df["injured"] = pd.to_datetime(cleaned_injury_df["injured"], errors="coerce")

# Extract year and compute age
cleaned_injury_df["age"] = cleaned_injury_df["injured"].dt.year - cleaned_injury_df["birth_date"].dt.year


In [35]:
drop_duplicated_df = cleaned_injury_df.drop_duplicates()
drop_duplicated_df.to_csv('/content/drop_duplicated_df.csv', index=False)

In [39]:
# Keep only rows with age between 18 and 40 (typical NBA career span)
drop_duplicated_df = drop_duplicated_df[(cleaned_injury_df["age"] >= 18) & (drop_duplicated_df["age"] <= 43)]


<ipython-input-39-444065f3f099>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  drop_duplicated_df = drop_duplicated_df[(cleaned_injury_df["age"] >= 18) & (drop_duplicated_df["age"] <= 43)]


In [40]:
# Parse injury dates
drop_duplicated_df["injured"] = pd.to_datetime(drop_duplicated_df["injured"], errors="coerce")

# Sort data by player and injury date
drop_duplicated_df = drop_duplicated_df.sort_values(by=["player_name", "injured"])

# Group by player and calculate cumulative count (excluding current injury)
drop_duplicated_df["prior_injuries"] = (
    drop_duplicated_df.groupby("player_name").cumcount()
)

In [41]:
drop_duplicated_df.to_csv('/content/drop_duplicated_df.csv', index=False)